In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
!pip install timm
import timm
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import os
from PIL import Image
from torch.utils.data import Dataset
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00


In [ ]:
#Öncelikle collabdan driveımıza bağlanmak için bir doğrulama yapmamız gerekiyor.
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [ ]:
#Burada ise erişimi uzun olan drive dizini için kısa bir yol tanıma yapıyoruz
# "/content/gdrive/My\ Drive/" olan dizine /mydrive diyerek ulaşabiliriz.
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'AraRapor .docx'			  'Colab Notebooks'	   Sunum
'Bitirme Projesi Cheklist.docx'		   coursera		  'teknofest 2024 '
'Bitirme Projesi Zaman Çizelgesi.gsheet'  'İngilizce kitabı pdf'  'teknofest başvuru'
'Büyük Dil Modellerine Giriş'		  'machine learning'	   yolov3
'C files'				   sertifika		   yolov4


In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/Büyük\ Dil\ Modellerine\ Giriş/HW2/data.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../data.zip -d data/

In [ ]:
root = "/data"
a = os.listdir(root)
dic = {}
csv_file = pd.DataFrame()
for disease in a:
    sub_path =  os.path.join(root,disease)
    for img in os.listdir(sub_path):
        dict_a = {'img': ['0'], 'label': ['0']}
        temp = pd.DataFrame(dict_a)

        temp.iloc[0,0] = os.path.join(sub_path,img)
        temp.iloc[0,1] = disease
        csv_file = pd.concat([csv_file,temp])

In [ ]:
csv_file

In [ ]:
dic = {}
for item in tqdm(set(csv_file["label"])):
    dic[item] = len(dic)
dic

In [ ]:
class PatchDataset(Dataset):
    def __init__(self, dic_map,file, mode = "train"):
        self.mode = mode
#         self.root = root
        if self.mode == "train":
            print("Train")
            self.file = file
            self.transformer = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(0.75, 1.33)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
#             self.transformer = transforms.Compose([
#             transforms.Resize((224,224)),
# #             transforms.RandomResizedCrop(224,scale=(0.5,1.s0)),
# #             .............add more transforms
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
        else:
            print("Test")
            self.file = file
            self.transformer = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

        self.dic_map = dic_map


    def __getitem__(self, item):
        path = self.file.iloc[item,0]
#         path = os.path.join(self.root+path)
        label = self.dic_map[self.file.iloc[item,1]]
        img = Image.open(path).convert('RGB')
        img = self.transformer(img)
        label = torch.tensor(label)

        return img, label

    def __len__(self):
        return len(self.file)

In [ ]:
from sklearn.model_selection import train_test_split
# Assuming trainFile is your original training DataFrame
train_df, test_df = train_test_split(csv_file, test_size=0.2, random_state=42)
dataset1 = PatchDataset(dic, train_df)
dataset2 = PatchDataset(dic, test_df, mode="test")

trainset = torch.utils.data.DataLoader(dataset1, batch_size=16, shuffle=True)
testset = torch.utils.data.DataLoader(dataset2, batch_size=16, shuffle=True)

Train
Test


In [ ]:
len(trainset)

23

In [ ]:
len(testset)


6

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss() #Loss
timm.list_models()

In [ ]:
net = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=7).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-5)

for epoch in range(10):
    net.train()
    loss_batch = 0
    for data, target in tqdm(trainset):
        data = data.to(device)
        target = target.to(device)
        output = net(data) #output
        loss = criterion(output, target) #output
        loss_batch+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("The {} epoch Loss is {}".format(epoch,loss_batch/len(trainset)))
    net.eval()
    correct = 0
    total = 0
    best_accuracy = 0.0
    with torch.no_grad():
        for data,target in tqdm(testset):
            data = data.to(device)
            target = target.to(device)
            output = net(data)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            #print(correct)
    accuracy = correct / len(testset.dataset)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(net.state_dict(),"best_model.pth")
        print("Saved model with accuracy: {:.2f}".format(best_accuracy))
    print("The {} epoch accuracy is {}".format(epoch,correct/len(testset.dataset)))

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

100%|██████████| 23/23 [11:42<00:00, 30.54s/it]


The 0 epoch Loss is 1.8520126861074697


100%|██████████| 6/6 [01:04<00:00, 10.81s/it]


...

Saved model with accuracy: 0.74
The 8 epoch accuracy is 0.7362637362637363


100%|██████████| 23/23 [11:24<00:00, 29.76s/it]


The 9 epoch Loss is 0.31011804743953375


100%|██████████| 6/6 [00:59<00:00, 10.00s/it]


Saved model with accuracy: 0.73
The 9 epoch accuracy is 0.7252747252747253


In [ ]:
print("Saved model with accuracy: {:.2f}".format(best_accuracy))
print("The Loss is: {}".format(loss))

Saved model with accuracy: 0.73
The Loss is: 0.2994900643825531
